In [1]:
# modules
from IPython.core.display import display, HTML
display(HTML("<style>.output_wrapper, .output {    height:auto !important;max-height:1000px;  /* your desired max-height here */}.output_scroll {    box-shadow:none !important;    webkit-box-shadow:none !important;}</style>"))

import random
import os
import re
import sys
import argparse
import subprocess
from time import clock
import pandas as pd
import numpy as np
from itertools import groupby
import itertools
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio.Alphabet import IUPAC, Gapped, generic_dna
from Bio.Align.Applications import ClustalwCommandline
from sys import argv
from Bio import AlignIO, SeqIO, Phylo
from time import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns; sns.set()
from scipy import stats
from random import randint,randrange


In [4]:
#%%notify


def status(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,number_of_generations):
    print("Recieved parameters: \n \t Ne: {Ne}, \n \t seq_len: {seq_len}, \n \t percent_of_recombinating_sp: {percent_of_recombinating_sp}, \n \t mutations_per_gen: {mutations_per_gen}, \n \t number_of_generations: {number_of_generations}".format(Ne = Ne,
seq_len = seq_len,
percent_of_recombinating_sp = percent_of_recombinating_sp,
mutations_per_gen = mutations_per_gen,
number_of_generations = number_of_generations))

#status(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,number_of_generations)  



# creating a model

def an_elementary_act_of_evolution(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,nuc_mode=None,population_mode=None):
    random.seed(666)
    #status(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,number_of_generations)
    
    if nuc_mode == "nuc":  # выбираем, с чем работаем
        seed = "ATGC"
    else:
        seed = "1234"
    
    # creating mutation function -- ok
    def mutate(seq, mutations_per_gen, mode=None):
        seq_cp = seq.copy()

        if mode == "new_population":
            mutations_per_gen_tmp = mutations_per_gen + int(0.01 * seq_len)
            for i in range(mutations_per_gen_tmp):
                mutation_site = randrange(len(seq_cp))
                seq_cp[mutation_site] = random.choice(seed)     
            return seq_cp 
        else:
            r = random.uniform(0, 1) # чтобы было меньше мутаций
            if r > 1/10:
                return seq_cp
            else:
                mutations_per_gen_tmp = mutations_per_gen
                for i in range(mutations_per_gen_tmp):
                    mutation_site = randrange(len(seq_cp))
                    seq_cp[mutation_site] = random.choice(seed)     
                return seq_cp
         
    if population_mode == "first":   # --ok
        # creating seqs
        seq = [random.choice(seed) for i in range(seq_len)]
        population = [mutate(seq, mutations_per_gen, "new_population") for x in range(Ne)]
    else:
        population = population_mode
        Ne = len(population)
    # breeding
    selected_list = []
    number_of_species = int(percent_of_recombinating_sp * len(population))
    if number_of_species%2 != 0:
        number_of_species -= 1
        
    for i in range(number_of_species): # --ok
        p = randrange(len(population))
        selected_list.append(population.pop(p))
    
    # crossingover
    def crossingover(seq_in1, seq_in2):
        number_of_parts = random.randrange(0, 2, 1)
        if number_of_parts > 0:   
            shuffled_1 = []
            shuffled_2 = []
            left = 0
            right = len(seq_in1)
            ind = True
            for i in range(number_of_parts):
                n = number_of_parts - i
                #print(right - left)
                if right - left > n + 1:
                    #print(i, left, right, ind)
                    
                    #print("right//n", right//n)
                    position = random.randint(left, right//n)
                    #print("position", position)
                    if ind == True:
                        shuffled_1.append(seq_in1[left:position])
                        shuffled_2.append(seq_in2[left:position])
                        #print(1, shuffled_1, "---", shuffled_2)
                        ind = False
                        left = position
                        continue
                    if ind == False:
                        shuffled_1.append(seq_in2[left:position])
                        shuffled_2.append(seq_in1[left:position])
                        #print(2, shuffled_1, "---", shuffled_2)
                        ind = True
                        left = position
                        continue
                else:
                    #print("out")
                    if ind == True:
                        shuffled_1.append(seq_in1[left:position])
                        shuffled_2.append(seq_in2[left:position])
                        #print(1, shuffled_1, "---", shuffled_2)
                    if ind == False:
                        shuffled_1.append(seq_in2[left:position])
                        shuffled_2.append(seq_in1[left:position])
                        #print(2, shuffled_1, "---", shuffled_2)
                    return shuffled_1, shuffled_2                        
            if ind == True:
                shuffled_1.append(seq_in1[left:])
                shuffled_2.append(seq_in2[left:])
                #print(1, shuffled_1, "---", shuffled_2)
            if ind == False:
                shuffled_1.append(seq_in2[left:])
                shuffled_2.append(seq_in1[left:])
                #print(2, shuffled_1, "---", shuffled_2)
            return shuffled_1, shuffled_2
        else:
            return seq_in1, seq_in2

    
    
    # multipy selected seqs
    def multiply_seqs(result, work_list, mutations_per_gen, mode=None):
        for i, j in zip(work_list[0::2], work_list[1::2]):
            if mode == "crossingover_it":
                
                p, q = crossingover(i, j)
                
                p = list(itertools.chain.from_iterable(p))
                q = list(itertools.chain.from_iterable(q))

                for k in range(1): 
                    result.append(mutate(p, mutations_per_gen))
                    result.append(mutate(q, mutations_per_gen))
            else:
                for k in range(1):
                    result.append(mutate(i, mutations_per_gen))
                    result.append(mutate(j, mutations_per_gen))
        return result
    
    result = []
    result = multiply_seqs(result, selected_list, mutations_per_gen, "crossingover_it")
    result = multiply_seqs(result, population, mutations_per_gen)
    
    result = random.sample(result, Ne)
    return result



def decoder(result):
    def make_it_in_nucl(seq):
        dic = {1:"A",2:"T",3:"G",4:"C"}
        for j in range(len(seq)):
            seq[j] = dic[int(seq[j])]
        return seq
    for i in range(len(result)):
        result[i] = make_it_in_nucl(result[i])
    return result


root = "/home/pavel/Desktop/Work/Project/Simulation"
outputdir = os.path.abspath(root + '/result_0.01')
    
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

    
# loading parameters
mutations_per_gen = 1
seq_len = 1000


os.chdir(outputdir)
for Ne in [x * 10000 for x in range(1, 10, 2)]: #+ [x * 1000 for x in range(2, 10, 2)]:
    print("--Ne:", Ne)
    for number_of_generations in [x * 10000 for x in range(1, 10, 2)]:# + [x * 1000 for x in range(2, 10, 2)]:
        print("--number_of_generations:", number_of_generations)
        for percent_of_recombinating_sp in [0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1]:
            print("--percent_of_recombinating_sp:", percent_of_recombinating_sp)
            print(" ")
            
            status(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,number_of_generations)  
            
            start = clock()
            step = an_elementary_act_of_evolution(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,"nuc","first")
            for i in range(number_of_generations):
                step = an_elementary_act_of_evolution(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,"nuc",step)
            end = clock()
            
            step = random.sample(step, 100) 
            
        
            def count_dict(mystring):
                d = {}
                for w in mystring: 
                    d[w] = mystring.count(w)
                #print(d.values(), min(d.values())/len(mystring))
                if min(d.values())/len(mystring) < 0.25:
                    return -1
                else:
                    return 1

            def slice_maker(step):
                for i in range(len(step[0])):
                    string = ""
                    for j in range(len(step)):
                        string += step[j][i]
                    #print(string)
                    if count_dict(string) == -1:
                        for j in range(len(step)):
                            step[j][i] = "N"
                    
            slice_maker(step)

            #step = decoder(step)
            #print(step)
            
            
            align = MultipleSeqAlignment([], Gapped(IUPAC.unambiguous_dna, "-"))
            for i in range(len(step)):
                align.add_sequence(str(i), "".join(step[i]))
            
            SeqIO.write(align, outputdir + "/" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + "_align.fasta", "fasta")
        
            print("timing is", round(end - start,5), "seconds")
            
            try:
                line1 = 'snp-sites -mvp -o {out_file} {in_file}'.format(out_file = outputdir + "/snp-sites_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len),
                    in_file = outputdir + "/" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + "_align.fasta")
                p = subprocess.Popen(line1, shell=True)
                p.wait()
            
                line2 = 'vcftools --hap-r2 --vcf {in_file} --out {out_file}'.format(in_file = outputdir + "/snp-sites_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".vcf", 
                        out_file = outputdir + "/vcftools_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len))
                p = subprocess.Popen(line2, shell=True)
                p.wait()
                
                    
                df = pd.read_csv(outputdir + "/vcftools_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".hap.ld", sep='\t',index_col=False)
                df["Len"] = df["POS2"] - df["POS1"]
                
                df_name = outputdir + "/LD_df_" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".csv"
                df.to_csv(path_or_buf = df_name)
                
                line3 = 'Rscript "/home/pavel/Desktop/Work/Project/Simulation/R_plot.R" --no-save --no-restore --args {outputdir} {plot_name} 0.0 {df_name}'.format(outputdir = outputdir, 
                plot_name = outputdir + "/Ne-" + str(Ne) + "_gen_sp-" + str(number_of_generations) + "_rec-" + str(percent_of_recombinating_sp) + "_mut-" + str(mutations_per_gen) + "_seq_l-" + str(seq_len),
                df_name = df_name)
                p = subprocess.Popen(line3, shell=True)
                p.wait()
                
                            
                line = 'rm {file}'.format(file = outputdir + "/snp-sites_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".phylip")
                p = subprocess.Popen(line, shell=True)

                line = 'rm {file}'.format(file = outputdir + "/snp-sites_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".snp_sites.aln")
                p = subprocess.Popen(line, shell=True)

                line = 'rm {file}'.format(file = outputdir + "/snp-sites_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".vcf")
                p = subprocess.Popen(line, shell=True)

                line = 'rm {file}'.format(file = outputdir + "/vcftools_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len))
                p = subprocess.Popen(line, shell=True)
                    
                line = 'rm {file}'.format(file = outputdir + "/vcftools_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".log")
                p = subprocess.Popen(line, shell=True)

                line = 'rm {file}'.format(file = outputdir + "/vcftools_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".hap.ld")
                p = subprocess.Popen(line, shell=True)
                    
                    
            except ZeroDivisionError:
                continue
        plt.close('all')

--Ne: 10000
--number_of_generations: 10000
--percent_of_recombinating_sp: 0
 
Recieved parameters: 
 	 Ne: 10000, 
 	 seq_len: 1000, 
 	 percent_of_recombinating_sp: 0, 
 	 mutations_per_gen: 1, 
 	 number_of_generations: 10000


KeyboardInterrupt: 

In [8]:
#%%notify


def status(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,number_of_generations):
    print("Recieved parameters: \n \t Ne: {Ne}, \n \t seq_len: {seq_len}, \n \t percent_of_recombinating_sp: {percent_of_recombinating_sp}, \n \t mutations_per_gen: {mutations_per_gen}, \n \t number_of_generations: {number_of_generations}".format(Ne = Ne,
seq_len = seq_len,
percent_of_recombinating_sp = percent_of_recombinating_sp,
mutations_per_gen = mutations_per_gen,
number_of_generations = number_of_generations))

#status(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,number_of_generations)  



# creating a model

def an_elementary_act_of_evolution(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,nuc_mode=None,population_mode=None):
    random.seed(666)
    #status(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,number_of_generations)
    
    if nuc_mode == "nuc":  # выбираем, с чем работаем
        seed = "ATGC"
    else:
        seed = "1234"
    
    # creating mutation function -- ok
    def mutate(seq, mutations_per_gen, mode=None):
        seq_cp = seq.copy()

        if mode == "new_population":
            mutations_per_gen_tmp = mutations_per_gen + int(0.01 * seq_len)
            for i in range(mutations_per_gen_tmp):
                mutation_site = randrange(len(seq_cp))
                seq_cp[mutation_site] = random.choice(seed)     
            return seq_cp 
        else:
            r = random.uniform(0, 1) # чтобы было меньше мутаций
            if r > 1/10:
                return seq_cp
            else:
                mutations_per_gen_tmp = mutations_per_gen
                for i in range(mutations_per_gen_tmp):
                    mutation_site = randrange(len(seq_cp))
                    seq_cp[mutation_site] = random.choice(seed)     
                return seq_cp
         
    if population_mode == "first":   # --ok
        # creating seqs
        seq = [random.choice(seed) for i in range(seq_len)]
        population = [mutate(seq, mutations_per_gen, "new_population") for x in range(Ne)]
    else:
        population = population_mode
        Ne = len(population)
    # breeding
    selected_list = []
    number_of_species = int(percent_of_recombinating_sp * len(population))
    if number_of_species%2 != 0:
        number_of_species -= 1
        
    for i in range(number_of_species): # --ok
        p = randrange(len(population))
        selected_list.append(population.pop(p))
    
    # crossingover
    def crossingover(seq_in1, seq_in2):
        number_of_parts = random.randrange(0, 2, 1)
        if number_of_parts > 0:   
            shuffled_1 = []
            shuffled_2 = []
            left = 0
            right = len(seq_in1)
            ind = True
            for i in range(number_of_parts):
                n = number_of_parts - i
                #print(right - left)
                if right - left > n + 1:
                    #print(i, left, right, ind)
                    
                    #print("right//n", right//n)
                    position = random.randint(left, right//n)
                    #print("position", position)
                    if ind == True:
                        shuffled_1.append(seq_in1[left:position])
                        shuffled_2.append(seq_in2[left:position])
                        #print(1, shuffled_1, "---", shuffled_2)
                        ind = False
                        left = position
                        continue
                    if ind == False:
                        shuffled_1.append(seq_in2[left:position])
                        shuffled_2.append(seq_in1[left:position])
                        #print(2, shuffled_1, "---", shuffled_2)
                        ind = True
                        left = position
                        continue
                else:
                    #print("out")
                    if ind == True:
                        shuffled_1.append(seq_in1[left:position])
                        shuffled_2.append(seq_in2[left:position])
                        #print(1, shuffled_1, "---", shuffled_2)
                    if ind == False:
                        shuffled_1.append(seq_in2[left:position])
                        shuffled_2.append(seq_in1[left:position])
                        #print(2, shuffled_1, "---", shuffled_2)
                    return shuffled_1, shuffled_2                        
            if ind == True:
                shuffled_1.append(seq_in1[left:])
                shuffled_2.append(seq_in2[left:])
                #print(1, shuffled_1, "---", shuffled_2)
            if ind == False:
                shuffled_1.append(seq_in2[left:])
                shuffled_2.append(seq_in1[left:])
                #print(2, shuffled_1, "---", shuffled_2)
            return shuffled_1, shuffled_2
        else:
            return seq_in1, seq_in2

    
    
    # multipy selected seqs
    def multiply_seqs(result, work_list, mutations_per_gen, mode=None):
        for i, j in zip(work_list[0::2], work_list[1::2]):
            if mode == "crossingover_it":
                
                p, q = crossingover(i, j)
                
                p = list(itertools.chain.from_iterable(p))
                q = list(itertools.chain.from_iterable(q))

                for k in range(1): 
                    result.append(mutate(p, mutations_per_gen))
                    result.append(mutate(q, mutations_per_gen))
            else:
                for k in range(1):
                    result.append(mutate(i, mutations_per_gen))
                    result.append(mutate(j, mutations_per_gen))
        return result
    
    result = []
    result = multiply_seqs(result, selected_list, mutations_per_gen, "crossingover_it")
    result = multiply_seqs(result, population, mutations_per_gen)
    
    result = random.sample(result, Ne)
    return result



def decoder(result):
    def make_it_in_nucl(seq):
        dic = {1:"A",2:"T",3:"G",4:"C"}
        for j in range(len(seq)):
            seq[j] = dic[int(seq[j])]
        return seq
    for i in range(len(result)):
        result[i] = make_it_in_nucl(result[i])
    return result


root = "/home/pavel/Desktop/Work/Project/Simulation"
outputdir = os.path.abspath(root + '/result_0.01')
    
if not os.path.exists(outputdir):
    os.makedirs(outputdir)

    
# loading parameters
mutations_per_gen = 1
seq_len = 1000


os.chdir(outputdir)
for Ne in [x * 5000 for x in range(1, 10, 2)]: #+ [x * 1000 for x in range(2, 10, 2)]:
    print("--Ne:", Ne)
    for number_of_generations in [x * 10000 for x in range(1, 10, 2)]:# + [x * 1000 for x in range(2, 10, 2)]:
        print("--number_of_generations:", number_of_generations)
        for percent_of_recombinating_sp in [x * 0.001 for x in range(1, 10, 2)] + [x * 0.01 for x in range(1, 10, 2)] + [x * 0.01 for x in range(1, 10, 2)] + [x * 0.1 for x in range(1, 10, 2)]:
            print("--percent_of_recombinating_sp:", percent_of_recombinating_sp)
            print(" ")
            
            status(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,number_of_generations)  
            
            start = clock()
            step = an_elementary_act_of_evolution(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,"nuc","first")
            for i in range(number_of_generations):
                step = an_elementary_act_of_evolution(Ne,seq_len,percent_of_recombinating_sp,mutations_per_gen,"nuc",step)
            end = clock()
            
            step = random.sample(step, 100) 
            
        
            def count_dict(mystring):
                d = {}
                for w in mystring: 
                    d[w] = mystring.count(w)
                #print(d.values(), min(d.values())/len(mystring))
                if min(d.values())/len(mystring) < 0.25:
                    return -1
                else:
                    return 1

            def slice_maker(step):
                for i in range(len(step[0])):
                    string = ""
                    for j in range(len(step)):
                        string += step[j][i]
                    #print(string)
                    if count_dict(string) == -1:
                        for j in range(len(step)):
                            step[j][i] = "N"
                    
            slice_maker(step)

            #step = decoder(step)
            #print(step)
            
            
            align = MultipleSeqAlignment([], Gapped(IUPAC.unambiguous_dna, "-"))
            for i in range(len(step)):
                align.add_sequence(str(i), "".join(step[i]))
            
            SeqIO.write(align, outputdir + "/" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + "_align.fasta", "fasta")
        
            print("timing is", round(end - start,5), "seconds")
            
            try:
                line1 = 'snp-sites -mvp -o {out_file} {in_file}'.format(out_file = outputdir + "/snp-sites_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len),
                    in_file = outputdir + "/" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + "_align.fasta")
                p = subprocess.Popen(line1, shell=True)
                p.wait()
            
                line2 = 'vcftools --hap-r2 --vcf {in_file} --out {out_file}'.format(in_file = outputdir + "/snp-sites_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".vcf", 
                        out_file = outputdir + "/vcftools_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len))
                p = subprocess.Popen(line2, shell=True)
                p.wait()
                
                    
                df = pd.read_csv(outputdir + "/vcftools_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".hap.ld", sep='\t',index_col=False)
                df["Len"] = df["POS2"] - df["POS1"]
                
                df_name = outputdir + "/LD_df_" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".csv"
                df.to_csv(path_or_buf = df_name)
                
                line3 = 'Rscript "/home/pavel/Desktop/Work/Project/Simulation/R_plot.R" --no-save --no-restore --args {outputdir} {plot_name} 0.0 {df_name}'.format(outputdir = outputdir, 
                plot_name = outputdir + "/Ne-" + str(Ne) + "_gen_sp-" + str(number_of_generations) + "_rec-" + str(percent_of_recombinating_sp) + "_mut-" + str(mutations_per_gen) + "_seq_l-" + str(seq_len),
                df_name = df_name)
                p = subprocess.Popen(line3, shell=True)
                p.wait()
                
                            
                line = 'rm {file}'.format(file = outputdir + "/snp-sites_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".phylip")
                p = subprocess.Popen(line, shell=True)

                line = 'rm {file}'.format(file = outputdir + "/snp-sites_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".snp_sites.aln")
                p = subprocess.Popen(line, shell=True)

                line = 'rm {file}'.format(file = outputdir + "/snp-sites_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".vcf")
                p = subprocess.Popen(line, shell=True)

                line = 'rm {file}'.format(file = outputdir + "/vcftools_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len))
                p = subprocess.Popen(line, shell=True)
                    
                line = 'rm {file}'.format(file = outputdir + "/vcftools_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".log")
                p = subprocess.Popen(line, shell=True)

                line = 'rm {file}'.format(file = outputdir + "/vcftools_out_file" + str(number_of_generations) + "_" + str(Ne) + "_" + str(percent_of_recombinating_sp) + "_" + str(mutations_per_gen) + "_" + str(seq_len) + ".hap.ld")
                p = subprocess.Popen(line, shell=True)
                    
                    
            except ZeroDivisionError:
                continue
        plt.close('all')

--Ne: 5000
--number_of_generations: 10000
--percent_of_recombinating_sp: 0.001
 
Recieved parameters: 
 	 Ne: 5000, 
 	 seq_len: 1000, 
 	 percent_of_recombinating_sp: 0.001, 
 	 mutations_per_gen: 1, 
 	 number_of_generations: 10000
timing is 687.60453 seconds
--percent_of_recombinating_sp: 0.005
 
Recieved parameters: 
 	 Ne: 5000, 
 	 seq_len: 1000, 
 	 percent_of_recombinating_sp: 0.005, 
 	 mutations_per_gen: 1, 
 	 number_of_generations: 10000
timing is 727.65623 seconds
--percent_of_recombinating_sp: 0.01
 
Recieved parameters: 
 	 Ne: 5000, 
 	 seq_len: 1000, 
 	 percent_of_recombinating_sp: 0.01, 
 	 mutations_per_gen: 1, 
 	 number_of_generations: 10000
timing is 803.02357 seconds
--percent_of_recombinating_sp: 0.02
 
Recieved parameters: 
 	 Ne: 5000, 
 	 seq_len: 1000, 
 	 percent_of_recombinating_sp: 0.02, 
 	 mutations_per_gen: 1, 
 	 number_of_generations: 10000
timing is 768.75202 seconds
--percent_of_recombinating_sp: 0.05
 
Recieved parameters: 
 	 Ne: 5000, 
 	 seq_l

KeyboardInterrupt: 